This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

#print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules

In [3]:
from urllib.request import urlopen
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [4]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

In [5]:
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-1-1&end_date=2017-12-31&api_key={}'.format(API_KEY)
data = json.loads(urlopen(url).read().decode('utf-8'))

In [6]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(data['dataset'].keys())
print(data['dataset']['column_names'])
print(data['dataset']['data'][0])

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])
['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]


In [7]:
DATE  = 0
OPEN  = 1
HIGH  = 2
LOW   = 3
CLOSE = 4
VOLUME = 6

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [8]:
highest_opening = None
lowest_opening = None
largest_change_in_one_day = None
largest_change_between_two_days = None
previous_closing_price = None

num_days = 0
sum_trading_volumes = 0
average_trading_volume = None
median_trading_volume = None

for quote in data['dataset']['data']:
    # Handle opening price
    if quote[OPEN] == None:
        print('Warning: Empty OPEN price: {}. Taking the mid value.'.format(quote))
        opening = (quote[HIGH] + quote[LOW]) / 2.0
    else:
        opening = quote[OPEN]

    if highest_opening == None or opening > highest_opening:
        highest_opening = opening

    if lowest_opening == None or opening < lowest_opening:
        lowest_opening = opening

    # Handle change in one day
    change_in_one_day = quote[HIGH] - quote[LOW]
    if largest_change_in_one_day == None or change_in_one_day > largest_change_in_one_day:
        largest_change_in_one_day = change_in_one_day

    # Handle change between two days
    if previous_closing_price == None:
        previous_closing_price = quote[CLOSE]
    else:
        change_between_two_days = quote[CLOSE] - previous_closing_price \
                                    if quote[CLOSE] > previous_closing_price \
                                    else previous_closing_price - quote[CLOSE]
        if largest_change_between_two_days == None or change_between_two_days > largest_change_between_two_days:
            largest_change_between_two_days = change_between_two_days

    # Handle trading volumes
    sum_trading_volumes += quote[VOLUME]
    num_days += 1

# Calculate average_trading_volume
average_trading_volume = sum_trading_volumes / num_days

# Calculate median_trading_volume
volumes = [ quote[VOLUME] for quote in data['dataset']['data']]
volumes.sort()
mid = int(len(volumes)/2)
if len(volumes) % 2 != 0:
    median_trading_volume = volumes[mid]
else:
    median_trading_volume = (volumes[mid - 1] + volumes[mid]) / 2.0

In [9]:
print('Highest open price:              {:.2f}'.format(highest_opening))
print('Lowest open price:               {:.2f}'.format(lowest_opening))
print('Largest change in one day:       {:.2f}'.format(largest_change_in_one_day))
print('Largest change between two days: {:.2f}'.format(largest_change_between_two_days))
print('Average trading volume:          {:.2f}'.format(average_trading_volume))
print('Median trading volume:           {:.2f}'.format(median_trading_volume))

Highest open price:              53.11
Lowest open price:               34.00
Largest change in one day:       2.81
Largest change between two days: 17.70
Average trading volume:          89124.34
Median trading volume:           76286.00
